In [5]:
import pandas as pd
import numpy as np
import torch
from torch import bfloat16
import transformers
from transformers import AutoTokenizer
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

import warnings
warnings.filterwarnings("ignore")

In [6]:
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(torch.cuda.current_device())
    total_memory = torch.cuda.get_device_properties(0).total_memory
    total_memory_gb = total_memory / (1024**3) # Converting memory to Gb
    print("GPU is available. \nUsing GPU")
    print("\nGPU Name:", gpu_name)
    print(f"Total GPU Memory: {total_memory_gb:.2f} GB")
    
    device = torch.device('cuda')
else:
    print("GPU is not available. \nUsing CPU")
    device = torch.device('cpu')

GPU is not available. 
Using CPU


In [9]:
data = pd.read_excel('data.xlsx')
data.head()

,Theme,Answer
0,Кто заплатит дивиденды: топ-3 акции декабря,"Сургутнефтегаз, привилегированные акции — хоро..."
1,Кто заплатит дивиденды: топ-3 акции декабря,Газпром нефть — 9 ноября совет директоров комп...
2,Кто заплатит дивиденды: топ-3 акции декабря,Магнит — совет директоров рекомендовал выплати...
3,Грошовые акции — стоит ли в них инвестировать,Грошовые акции — это акции небольших или пробл...
4,Грошовые акции — стоит ли в них инвестировать,Грошовые акции могут торговаться как на крупне...


In [10]:
answer = DataFrameLoader(data, page_content_column='Theme')

In [12]:
document = answer.load()

In [14]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=20)
splitted_texts = splitter.split_documents(document)

In [17]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 588kB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 94.9kB/s]
README.md: 100%|██████████| 10.7k/10.7k [00:00<00:00, 5.38MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 116kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 3.02MB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:36<00:00, 2.47MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 26.5kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 56.0kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.00MB/s]
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 175kB/s]
train_script.py: 100%|██████████| 13.2k/13.2k [00:00<00:00, 4.39MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 765kB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 174kB/s]


In [18]:
chroma_database = Chroma.from_documents(splitted_texts,
                                      embedding_model,
                                      persist_directory = 'chroma_db')

RuntimeError: [91mYour system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0.[0m
[94mPlease visit https://docs.trychroma.com/troubleshooting#sqlite to learn how to upgrade.[0m

In [19]:
import sqlite3
sqlite3.version

'2.6.0'

In [20]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

ModuleNotFoundError: No module named 'pysqlite3'